In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import os

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
beautiful_fpl_data = pd.read_csv('/content/drive/My Drive/STA221 Data/beautiful_fpl_data.csv', index_col=0)
merged_data = pd.read_csv('/content/drive/My Drive/STA221 Data/merged_fpl_dataset.csv', index_col=0)
merged_data = merged_data.rename(columns={'simpler-match-name':'match-name'})
#deal with nans
#based on: fpl_data.isna().sum().sort_values().tail(15)
print(merged_data.isna().sum().sort_values().tail(15))
merged_data.loc[:, 'Age'] = merged_data.Age.fillna(merged_data.Age.mode(dropna=True))
merged_data.loc[:, 'Nation'] = merged_data.Nation.fillna('Unknown')
#Rest of the nans are in match stats which can safely be set to 0
merged_data.fillna(0, inplace=True)
print(f'After Processing there are: {merged_data.isna().sum().sum()} NaNs')

Performance_PKcon       0
Performance_OG          0
Aerial Duels_Lost       0
Performance_Off         0
Age                     6
Nation                 13
Passes_Cmp%           137
Total_Cmp%            137
Short_Cmp%            391
Medium_Cmp%           599
Long_Cmp%            2344
Aerial Duels_Won%    3787
Challenges_Tkl%      5062
Take-Ons_Succ%       5288
Take-Ons_Tkld%       5288
dtype: int64
After Processing there are: 0 NaNs


In [ ]:
from pprint import pprint
common_cols = set(beautiful_fpl_data.columns).intersection(set(merged_data.columns))
missing_in_merged = set(beautiful_fpl_data.columns) - set(merged_data.columns) # = 0
missing_in_fpl = set(merged_data.columns) - set(beautiful_fpl_data.columns)
pprint(dict(common_cols=common_cols, missing_in_merged=missing_in_merged, missing_in_fpl=missing_in_fpl))

{'common_cols': {'GW',
                 'assists',
                 'bonus',
                 'bps',
                 'clean_sheets',
                 'creativity',
                 'element',
                 'expected_assists',
                 'expected_goal_involvements',
                 'expected_goals',
                 'expected_goals_conceded',
                 'fixture',
                 'fixture_code',
                 'fixture_event',
                 'fixture_finished',
                 'fixture_finished_provisional',
                 'fixture_id',
                 'fixture_kickoff_time',
                 'fixture_minutes',
                 'fixture_provisional_start_time',
                 'fixture_pulse_id',
                 'fixture_started',
                 'fixture_stats',
                 'fixture_team_a',
                 'fixture_team_a_difficulty',
                 'fixture_team_a_score',
                 'fixture_team_h',
                 'fixture_team_h_difficu

In [ ]:
# all of the players in merged data are also in beautiful_fpl_data
set(merged_data.name.unique()) - set(beautiful_fpl_data.name.unique())

set()

In [ ]:
merged_data['merger_col'] = merged_data.name + '---' + merged_data['match-name']
beautiful_fpl_data['merger_col'] = beautiful_fpl_data.name + '---' + beautiful_fpl_data['match-name']
fpl_keep_cols = list(set(beautiful_fpl_data.columns).intersection(set(merged_data.columns))) # common cols
merged_keep_cols = ['merger_col'] + list(set(merged_data.columns) - set(beautiful_fpl_data.columns)) # cols not in fpl
assert len(set(beautiful_fpl_data.columns) - set(merged_data.columns)) == 0, str(set(merged_data.columns) - set(beautiful_fpl_data.columns)) # no columns in fpl that are not in merged
assert set(fpl_keep_cols + merged_keep_cols) == set(list(beautiful_fpl_data.columns) + list(merged_data.columns)) # everything is kept

final_data = pd.merge(
    beautiful_fpl_data[fpl_keep_cols],
    merged_data[merged_keep_cols],
    how='left',
    on='merger_col',
    indicator=True
)
print(final_data._merge.value_counts())
final_data = final_data.drop(columns=['_merge'])

teams_data = pd.read_csv('https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/refs/heads/master/data/2024-25/teams.csv')
mapping_teams = teams_data.set_index('id')['name']
# used for verification
# (final_data['match-name'] + ' : ' + final_data.team + ' vs ' + final_data.opponent_team.map(mapping_teams)).iloc[0]
final_data.opponent_team = final_data.opponent_team.map(mapping_teams)

columns_to_remove = [
    '#', '1/3', 'Home', 'Player', 'element',
  'team_a_score',
  'team_h_score', 'transfers_balance', 'kickoff_time', 'Min', 'modified',
    'own_goals',
                 'penalties_missed',
                 'penalties_saved',
                 'red_cards',
                 'round',
                 'saves',
                 'starts',
                 'team_a_score',
                 'team_h_score',
                 'transfers_balance',
                 'yellow_cards'
]
meta = [
    'merger_col', 'was_home', 'GW', 'Age', 'Nation', 'match-name', 'mapping_name', 'name', 'position', 'Pos', 'Away', 'team', 'opponent_team'
] + [x for x in final_data.columns if x.startswith('fixture')]
target = ['total_points', 'bonus', 'bps', 'minutes']
available_features = ['xP', 'value', 'creativity', 'influence', 'threat', 'ict_index', 'transfers_out', 'transfers_in', 'selected', 'was_home']
features = [x for x in final_data.columns if x not in (meta + columns_to_remove + target + available_features)] #minutes is added as a feature seperately later
#rest should be match data


_merge
left_only     16814
both          10791
right_only        0
Name: count, dtype: int64


In [ ]:
print('GW' in features)

# ----for home team now ----
features = [x for x in features if (not x.startswith('opponent_team_mean_')) and (not x.startswith('team_mean_'))]
team_meaned_data = final_data.groupby(['team', 'GW'])[features].transform('mean').rename(columns={x:f'team_mean_{x}' for x in features})
print(team_meaned_data.columns)
for col in team_meaned_data.columns:
  if col.startswith('team_mean_'):
    final_data[col] = team_meaned_data[col]
    features.append(col)

    # ---- for opponent now ----
    final_data[f'opponent_{col}'] = 0.0
    features.append(f'opponent_{col}')

for team in final_data['team'].unique():
  for gameweek in final_data['GW'].unique():
    indexer = (final_data.GW == gameweek) & (final_data.team == team)
    opp_cols = [f'opponent_{col}' for col in team_meaned_data.columns if col.startswith('team_mean')]
    t_cols = [col for col in team_meaned_data.columns if col.startswith('team_mean')]
    if indexer.sum() > 0:
      vall = final_data[indexer][t_cols].iloc[0].copy(deep=True)
      for col in t_cols:
        final_data.loc[indexer, f'oppponent_{col}'] = vall[col]
      print(vall.min(), vall.max(), vall.isna().sum())
    else:
      print(f'No {team} in GW {gameweek}')
len(features), features

False
Index(['team_mean_expected_goal_involvements', 'team_mean_assists',
       'team_mean_expected_assists', 'team_mean_expected_goals',
       'team_mean_goals_scored', 'team_mean_clean_sheets',
       'team_mean_goals_conceded', 'team_mean_expected_goals_conceded',
       'team_mean_Performance_Sh', 'team_mean_Pass Types_TB',
       ...
       'team_mean_Long_Cmp', 'team_mean_Carries_CPA', 'team_mean_xAG',
       'team_mean_Pass Types_Sw', 'team_mean_Pass Types_FK',
       'team_mean_Expected_npxG', 'team_mean_Performance_Crs', 'team_mean_xA',
       'team_mean_Int', 'team_mean_Performance_CrdR'],
      dtype='object', length=116)


/tmp/ipython-input-1064828892.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_data[col] = team_meaned_data[col]
/tmp/ipython-input-1064828892.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_data[f'opponent_{col}'] = 0.0
/tmp/ipython-input-1064828892.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

0.0 413.4 0
0.0 310.92857142857144 0
0.0 387.6 0
0.0 270.93333333333334 0
0.0 441.1333333333333 0
0.0 416.7142857142857 0
0.0 452.93333333333334 0
0.0 417.06666666666666 0
0.0 359.8 0
0.0 415.0 0
0.0 381.93333333333334 0
0.0 533.5333333333333 0
0.0 339.2857142857143 0
0.0 246.06666666666666 0
0.0 481.6666666666667 0
0.0 488.57142857142856 0
0.0 311.6 0
0.0 538.5714285714286 0
0.0 480.5 0
0.0 468.7857142857143 0
0.0 390.53846153846155 0
0.0 313.64285714285717 0
0.0 380.85714285714283 0
0.0 467.61538461538464 0
0.0 490.1333333333333 0
0.0 383.73333333333335 0
0.0 512.1428571428571 0
0.0 322.42857142857144 0
0.0 462.46666666666664 0
0.0 571.7857142857143 0
0.0 332.8 0
0.0 365.61538461538464 0
0.0 625.2142857142857 0
0.0 236.85714285714286 0
0.0 465.5 0
0.0 526.4 0
0.0 482.07142857142856 0
0.0 484.2 0
0.0 432.0 0
0.0 336.35714285714283 0
0.0 364.7692307692308 0
0.0 232.46153846153845 0
0.0 204.28571428571428 0
0.0 270.7857142857143 0
0.0 279.61538461538464 0
0.0 428.38461538461536 0
0.0 29

(348,
 ['expected_goal_involvements',
  'assists',
  'expected_assists',
  'expected_goals',
  'goals_scored',
  'clean_sheets',
  'goals_conceded',
  'expected_goals_conceded',
  'Performance_Sh',
  'Pass Types_TB',
  'Blocks_Pass',
  'Total_Att',
  'Tackles_Att 3rd',
  'Aerial Duels_Lost',
  'Performance_Recov',
  'Touches_Att Pen',
  'Performance_Gls',
  'Take-Ons_Tkld%',
  'Ast',
  'Performance_PKcon',
  'Performance_PKwon',
  'Outcomes_Off',
  'Take-Ons_Succ%',
  'Pass Types_CK',
  'Total_TotDist',
  'Long_Att',
  'Pass Types_TI',
  'Long_Cmp%',
  'Performance_Ast',
  'Touches_Att 3rd',
  'Short_Att',
  'Corner Kicks_Out',
  'Outcomes_Blocks',
  'Performance_SoT',
  'Medium_Cmp',
  'Aerial Duels_Won%',
  'Touches_Live',
  'Carries_TotDist',
  'Total_Cmp%',
  'Touches_Def Pen',
  'Receiving_PrgR',
  'Touches_Mid 3rd',
  'Take-Ons_Succ',
  'Corner Kicks_Str',
  'Clr',
  'Performance_TklW',
  'Short_Cmp%',
  'Touches_Touches',
  'Total_PrgDist',
  'Tackles_Tkl',
  'Tackles_Def 3rd',


In [ ]:
def exponential_decay(delta_time, tau=pd.Timedelta(days=7)):
    """
    delta_time: pd.Timedelta or numeric (in same units as tau)
    tau: decay constant
    returns: float weight between 0 and 1
    """
    return np.exp(-delta_time / tau)

def fast_previous_n_days(
    data: pd.DataFrame,
    cols: list,
    init_val : list = None, # per col init val
    cutoff_time: pd.Timedelta = pd.Timedelta(days=365), #default is for last year
    decay_tau : pd.Timedelta = None #No time decay if set to None (simple average)
  ) -> pd.DataFrame:
  """
    Compute the average (optionally decayed) values of specified columns over
    previous matches within a given time cutoff for each player.

    This function works **in-place** on the input DataFrame, adding new columns
    for each input column representing the average of previous matches within
    `cutoff_time`, optionally applying exponential time decay.

    Parameters
    ----------
    data : pd.DataFrame
        Input DataFrame containing at least the columns 'name', 'kickoff_time',
        and the columns specified in `cols`.
    cols : list of str
        Column names to compute the previous-match statistics for.
    init_val : list or np.ndarray, optional
        Initial values for each column when no previous matches exist. If None,
        defaults to 0.0 for each column.
    cutoff_time : pd.Timedelta, optional
        Maximum time difference to consider for previous matches. Defaults to 365 days.
    decay_tau : pd.Timedelta, optional
        Time constant for exponential decay.
        You can interpret it as:
          After decay_tau days have passed the value is scaled to half of its orignal value (half-life).
        If None, no decay is applied and a
        simple average of previous matches within `cutoff_time` is used.


    Returns
    -------
    pd.DataFrame
        The original DataFrame with new columns added for each input column.
        Each new column is named `{col}_last_{cutoff_time.days}_days`.

    Notes
    -----
    - The function assumes `kickoff_time` is in datetime format; it will convert it
      if necessary.
    - Time decay is applied per previous match as:
          weight = exp(-(current_time - previous_time) / decay_tau)
      If `decay_tau` is None, weight defaults to 1.
    - The function preserves the original row order and indices.
    """

  if init_val is None:
    init_val = np.array([0.0 for col in cols])
  elif isinstance(init_val, list):
    init_val = np.array(init_val)
  assert (int(init_val.shape[0]) == len(cols)), f"invalid init values! {init_val.shape[0]}, {len(cols)}"

  # ensure kickoff_time is in the correct format
  data.kickoff_time = pd.to_datetime(data.kickoff_time)
  kept_cols = list(set(cols + ['name', 'kickoff_time']))
  grouped = data[kept_cols].groupby('name')

  #calculations happen here
  time_indices = grouped.kickoff_time.apply(lambda x: [(x < y) & ((y - x) < cutoff_time) for y in x])
  decay_vectors = grouped.kickoff_time.apply(lambda x: [exponential_decay((y-x), decay_tau) if decay_tau is not None else 1.0 for y in x])
  result = grouped[cols].apply(lambda g: [(g[y].multiply(decay[y], axis='index')).mean(axis=0) if y.sum() else init_val for (y,decay) in zip(time_indices[g.name], decay_vectors[g.name])])

  #explode results into a nice column with same indices to add to orignal data
  aligned_list = []
  for name, values in result.items():
      # Get the index of rows for this group
      idx = grouped.get_group(name).index
      # Create a Series with the computed values and correct index
      s = pd.DataFrame(values, columns=cols, index=idx)
      aligned_list.append(s)
  aligned_series = pd.concat(aligned_list).sort_index()

  #add it to the data inplace
  for col in cols:
    new_col = f'{col}_last_{cutoff_time.components.days}_days'
    data[new_col] = aligned_series[col]
  return data

In [ ]:
final_data[[x for x in final_data.columns if 'opponent' in x ]].describe()

,opponent_team_mean_expected_goal_involvements,opponent_team_mean_assists,opponent_team_mean_expected_assists,opponent_team_mean_expected_goals,opponent_team_mean_goals_scored,opponent_team_mean_clean_sheets,opponent_team_mean_goals_conceded,opponent_team_mean_expected_goals_conceded,opponent_team_mean_Performance_Sh,opponent_team_mean_Pass Types_TB,...,opponent_team_mean_Long_Cmp,opponent_team_mean_Carries_CPA,opponent_team_mean_xAG,opponent_team_mean_Pass Types_Sw,opponent_team_mean_Pass Types_FK,opponent_team_mean_Expected_npxG,opponent_team_mean_Performance_Crs,opponent_team_mean_xA,opponent_team_mean_Int,opponent_team_mean_Performance_CrdR
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_final_data = final_data.drop(columns=columns_to_remove)
final_final_data[features + ['minutes']] = final_final_data[features + ['minutes']].fillna(0.0)
final_final_data = final_final_data.rename(columns={x:f'meta_{x}' for x in meta})
final_final_data = final_final_data.rename(columns={x:f'target_{x}' for x in target})
final_final_data = final_final_data.rename(columns={x:f'feature_{x}' for x in features})
final_final_data['feature_minutes'] = final_final_data.target_minutes.copy(deep=True)

from pandas.api.types import is_numeric_dtype
print([x for x in final_final_data.columns if x.startswith('feature_') and not is_numeric_dtype(final_final_data[x])])

print(final_final_data.meta_fixture_kickoff_time.max())

print('Calculating 1000 day decay')
final_final_data = final_final_data.copy(deep=True)
final_final_data = final_final_data.rename(columns={'meta_fixture_kickoff_time': 'kickoff_time', 'meta_name':'name'})
fast_previous_n_days(final_final_data, [x for x in final_final_data.columns if x.startswith('feature_')], init_val=None, cutoff_time=pd.Timedelta(days=1000), decay_tau=pd.Timedelta(days=14))
final_final_data = final_final_data.rename(columns={'kickoff_time': 'meta_fixture_kickoff_time', 'name':'meta_name'})

print('Calculating 31 day decay')
final_final_data = final_final_data.copy(deep=True)
final_final_data = final_final_data.rename(columns={'meta_fixture_kickoff_time': 'kickoff_time', 'meta_name':'name'})
fast_previous_n_days(final_final_data, [x for x in final_final_data.columns if (x.startswith('feature_') and (not x.endswith('1000_days')))], init_val=None, cutoff_time=pd.Timedelta(days=31), decay_tau=pd.Timedelta(days=31))
final_final_data = final_final_data.rename(columns={'kickoff_time': 'meta_fixture_kickoff_time', 'name':'meta_name'})
final_final_data = final_final_data.copy(deep=True)

print('Adding pre match available features')
for col in available_features:
  final_final_data[col] = final_data[col]
final_final_data = final_final_data.copy(deep=True)

[]
2025-05-25T15:00:00Z
Calculating 1000 day decay


/tmp/ipython-input-193007219.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col] = aligned_series[col]
/tmp/ipython-input-193007219.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col] = aligned_series[col]
/tmp/ipython-input-193007219.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

Calculating 31 day decay


/tmp/ipython-input-193007219.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col] = aligned_series[col]
/tmp/ipython-input-193007219.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[new_col] = aligned_series[col]
/tmp/ipython-input-193007219.py:89: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe =

Adding pre match available features


In [ ]:
final_final_final_data = final_final_data.drop(columns=[x for x in final_final_data.columns if (x.startswith('feature_') and (not x.endswith('_days')) and (x not in available_features))])
#fixing dumb mistake
final_final_final_data = final_final_final_data.drop(columns=[x for x in final_final_final_data.columns if (('last_1000_days' in x) and ('last_31_days' in x))])
final_final_final_data = final_final_final_data.rename(columns={x:f'feature_pregame_{x}' for x in available_features})
len(final_final_final_data.columns), len(final_final_data.columns)

(627, 860)

In [ ]:
def build_hierarchy(strings, max_depth=None):
    root = {}

    for s in strings:
        parts = s.split("_")

        # Enforce max depth
        if max_depth is not None and len(parts) > max_depth:
            parts = parts[:max_depth - 1] + ["_".join(parts[max_depth - 1:])]

        current = root

        for part in parts:
            current = current.setdefault(part, {})

    return root

hierarchy = build_hierarchy(list(final_final_final_data.columns), max_depth=3)
pprint(hierarchy)

{'feature': {'Aerial Duels': {'Lost_last_1000_days': {},
                              'Lost_last_31_days': {},
                              'Won%_last_1000_days': {},
                              'Won%_last_31_days': {},
                              'Won_last_1000_days': {},
                              'Won_last_31_days': {}},
             'Ast': {'last_1000_days': {}, 'last_31_days': {}},
             'Att': {'last_1000_days': {}, 'last_31_days': {}},
             'Blocks': {'Blocks_last_1000_days': {},
                        'Blocks_last_31_days': {},
                        'Pass_last_1000_days': {},
                        'Pass_last_31_days': {},
                        'Sh_last_1000_days': {},
                        'Sh_last_31_days': {}},
             'Carries': {'1/3_last_1000_days': {},
                         '1/3_last_31_days': {},
                         'CPA_last_1000_days': {},
                         'CPA_last_31_days': {},
                         'Carries_la

In [ ]:
final_final_final_data.sort_values(by='target_total_points',ascending=False)[[x for x in final_final_final_data.columns if x.startswith('feature_')]+['target_total_points']]

,feature_pregame_transfers_out,feature_pregame_ict_index,feature_pregame_selected,feature_pregame_influence,feature_pregame_creativity,feature_pregame_threat,feature_pregame_value,feature_pregame_transfers_in,feature_pregame_xP,feature_goals_conceded_last_1000_days,...,feature_team_mean_Corner Kicks_Str_last_31_days,feature_team_mean_Challenges_Lost_last_31_days,feature_team_mean_Long_Cmp%_last_31_days,feature_team_mean_Passes_Cmp_last_31_days,feature_team_mean_KP_last_31_days,feature_team_mean_Performance_PKatt_last_31_days,feature_team_mean_SCA_GCA_last_31_days,feature_minutes_last_31_days,feature_pregame_was_home,target_total_points
3224,267123,28.6,4079413,149.0,54.7,82.0,106,223471,10.5,0.085587,...,0.000000,0.314583,30.630806,18.673094,0.423560,0.000000,0.122623,48.457011,True,25
8353,599,23.7,8086,128.6,13.1,95.0,51,155,6.2,0.046079,...,0.000000,0.383522,19.312047,11.955713,0.311190,0.000000,0.091902,8.583492,True,23
14660,31985,20.6,349371,122.0,38.5,45.0,55,22257,0.0,0.044780,...,0.022776,0.503500,23.664577,13.998225,0.428749,0.019242,0.130886,34.124614,False,22
15101,3185,21.5,22820,110.2,2.7,102.0,50,6753,7.5,0.079529,...,0.018112,0.451241,19.905134,12.951511,0.475766,0.010854,0.150007,42.582079,True,21
16526,924,0.0,2537,0.0,0.0,0.0,11,507,6.5,0.000000,...,0.000000,0.406855,22.806245,19.907209,0.298421,0.000000,0.000000,0.000000,True,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16059,12069,1.1,351969,9.6,1.2,0.0,42,6254,0.0,0.062549,...,0.000000,0.201785,19.741772,17.253914,0.301896,0.007286,0.106702,36.902165,False,-3
23137,4073,0.0,113801,0.0,0.2,0.0,42,748,-1.7,0.045971,...,0.000000,0.229208,19.746584,8.851296,0.323386,0.000000,0.139197,18.075073,True,-4
8719,2540,0.6,88889,5.6,0.0,0.0,40,13261,-1.0,0.196191,...,0.000000,0.376549,23.622214,18.219606,0.239990,0.017118,0.071630,55.843835,True,-4
8662,1892,0.2,24013,1.6,0.3,0.0,44,2010,0.8,0.094990,...,0.000000,0.251306,27.590688,16.602078,0.281009,0.000000,0.248043,26.417101,False,-4


In [ ]:
final_final_final_data.to_csv('/content/drive/My Drive/STA221 Data/final_dataset_with_teams_and_opponents.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
target, list(sorted(features))

(['total_points', 'bonus', 'bps'],
 ['Aerial Duels_Lost',
  'Aerial Duels_Won',
  'Aerial Duels_Won%',
  'Ast',
  'Att',
  'Away',
  'Blocks_Blocks',
  'Blocks_Pass',
  'Blocks_Sh',
  'Carries_1/3',
  'Carries_CPA',
  'Carries_Carries',
  'Carries_Dis',
  'Carries_Mis',
  'Carries_PrgC',
  'Carries_PrgDist',
  'Carries_TotDist',
  'Challenges_Att',
  'Challenges_Lost',
  'Challenges_Tkl',
  'Challenges_Tkl%',
  'Clr',
  'Corner Kicks_In',
  'Corner Kicks_Out',
  'Corner Kicks_Str',
  'CrsPA',
  'Err',
  'Expected_npxG',
  'Expected_xAG',
  'Expected_xG',
  'GW',
  'Int',
  'KP',
  'Long_Att',
  'Long_Cmp',
  'Long_Cmp%',
  'Medium_Att',
  'Medium_Cmp',
  'Medium_Cmp%',
  'Min',
  'Outcomes_Blocks',
  'Outcomes_Cmp',
  'Outcomes_Off',
  'PPA',
  'Pass Types_CK',
  'Pass Types_Crs',
  'Pass Types_Dead',
  'Pass Types_FK',
  'Pass Types_Live',
  'Pass Types_Sw',
  'Pass Types_TB',
  'Pass Types_TI',
  'Passes_Att',
  'Passes_Cmp',
  'Passes_Cmp%',
  'Passes_PrgP',
  'Performance_2CrdY',
 

In [ ]:
final_data.isna().sum()[final_data.isna().sum() > 0].index

Index(['Touches_Def 3rd', 'Pass Types_TI', 'Performance_Int',
       'Performance_Off', 'Player', 'Performance_Blocks', 'Tackles_TklW',
       'Pass Types_Crs', 'Performance_PKwon', 'Performance_Sh',
       ...
       'xA', 'Take-Ons_Att', 'Performance_CrdR', 'Pass Types_CK',
       'Expected_xG', 'Blocks_Sh', 'Performance_CrdY', 'Carries_Carries',
       'Take-Ons_Succ', 'Receiving_Rec'],
      dtype='object', length=118)

In [ ]:
pprint(sorted(list(zip(final_data.columns, final_data.dtypes))))

[('#', dtype('float64')),
 ('1/3', dtype('float64')),
 ('Aerial Duels_Lost', dtype('float64')),
 ('Aerial Duels_Won', dtype('float64')),
 ('Aerial Duels_Won%', dtype('float64')),
 ('Age', dtype('O')),
 ('Ast', dtype('float64')),
 ('Att', dtype('float64')),
 ('Away', dtype('O')),
 ('Blocks_Blocks', dtype('float64')),
 ('Blocks_Pass', dtype('float64')),
 ('Blocks_Sh', dtype('float64')),
 ('Carries_1/3', dtype('float64')),
 ('Carries_CPA', dtype('float64')),
 ('Carries_Carries', dtype('float64')),
 ('Carries_Dis', dtype('float64')),
 ('Carries_Mis', dtype('float64')),
 ('Carries_PrgC', dtype('float64')),
 ('Carries_PrgDist', dtype('float64')),
 ('Carries_TotDist', dtype('float64')),
 ('Challenges_Att', dtype('float64')),
 ('Challenges_Lost', dtype('float64')),
 ('Challenges_Tkl', dtype('float64')),
 ('Challenges_Tkl%', dtype('float64')),
 ('Clr', dtype('float64')),
 ('Corner Kicks_In', dtype('float64')),
 ('Corner Kicks_Out', dtype('float64')),
 ('Corner Kicks_Str', dtype('float64')),
 ('

In [ ]:
final_data.select_dtypes(include=['object']).columns

Index(['merger_col', 'fixture_kickoff_time', 'position', 'team', 'name',
       'kickoff_time', 'match-name', 'fixture_stats', 'fixture_team_h',
       'fixture_team_a', 'Player', 'Home', 'mapping_name', 'Away', 'Age',
       'Pos', 'Nation'],
      dtype='object')

In [ ]:
beautiful_fpl_data.name

,name
0,Alex Scott
1,Carlos Miguel dos Santos Pereira
2,Tomiyasu Takehiro
3,Malcolm Ebiowei
4,Ben Brereton Díaz
...,...
27600,Ashley Young
27601,Somto Boniface
27602,Simon Rusk
27603,Arne Slot
